<a href="https://colab.research.google.com/github/HusseinMansourMohd/Data_Minning_final_project/blob/main/Classifying_with_FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. multilayer perceptron (MLP)

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

#load FashionMNIST dataset
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=train_transforms)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=test_transform)
train_size = int(0.8 * len(trainset))
validation_size = len(trainset) - train_size
train_dataset, validation_dataset = random_split(trainset, [train_size, validation_size])
trainloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
validationloader = DataLoader(validation_dataset, batch_size=4, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)


100%|██████████| 26421880/26421880 [00:01<00:00, 14357366.57it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272817.17it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5033136.12it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 19683023.69it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [ ]:

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.layer_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer_stack(x)
        return x



#enable GPU
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


#train
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')





Using device: cuda:0
[1,  2000] loss: 1.312
[1,  4000] loss: 0.770
[1,  6000] loss: 0.662
[1,  8000] loss: 0.625
[1, 10000] loss: 0.587
[1, 12000] loss: 0.556
[2,  2000] loss: 0.527
[2,  4000] loss: 0.523
[2,  6000] loss: 0.511
[2,  8000] loss: 0.500
[2, 10000] loss: 0.499
[2, 12000] loss: 0.496
[3,  2000] loss: 0.486
[3,  4000] loss: 0.471
[3,  6000] loss: 0.451
[3,  8000] loss: 0.472
[3, 10000] loss: 0.461
[3, 12000] loss: 0.438
[4,  2000] loss: 0.446
[4,  4000] loss: 0.431
[4,  6000] loss: 0.427
[4,  8000] loss: 0.435
[4, 10000] loss: 0.446
[4, 12000] loss: 0.412
[5,  2000] loss: 0.418
[5,  4000] loss: 0.434
[5,  6000] loss: 0.413
[5,  8000] loss: 0.409
[5, 10000] loss: 0.391
[5, 12000] loss: 0.407
[6,  2000] loss: 0.396
[6,  4000] loss: 0.395
[6,  6000] loss: 0.413
[6,  8000] loss: 0.396
[6, 10000] loss: 0.383
[6, 12000] loss: 0.390
[7,  2000] loss: 0.395
[7,  4000] loss: 0.378
[7,  6000] loss: 0.383
[7,  8000] loss: 0.390
[7, 10000] loss: 0.374
[7, 12000] loss: 0.373
[8,  2000] lo

In [ ]:
model.eval()
from sklearn.metrics import classification_report, accuracy_score

y_pred= []
y_true=[]
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')

print(classification_report(y_true, y_pred, target_names=testset.classes, digits=3))


              precision    recall  f1-score   support

 T-shirt/top      0.113     0.894     0.201      1000
     Trouser      0.000     0.000     0.000      1000
    Pullover      0.000     0.000     0.000      1000
       Dress      0.000     0.000     0.000      1000
        Coat      0.000     0.000     0.000      1000
      Sandal      0.501     0.650     0.566      1000
       Shirt      0.000     0.000     0.000      1000
     Sneaker      0.457     0.368     0.408      1000
         Bag      0.000     0.000     0.000      1000
  Ankle boot      0.000     0.000     0.000      1000

    accuracy                          0.191     10000
   macro avg      0.107     0.191     0.117     10000
weighted avg      0.107     0.191     0.117     10000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

# 2. Support Vector Machine

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

X_train = trainset.data.numpy().reshape((trainset.data.shape[0], -1))
y_train = trainset.targets.numpy()
X_test = testset.data.numpy().reshape((testset.data.shape[0], -1))
y_test = testset.targets.numpy()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

y_pred_val = clf.predict(X_val)
print(classification_report(y_val, y_pred_val))

# Test svm
y_pred_test = clf.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84      1202
           1       0.99      0.97      0.98      1219
           2       0.83      0.83      0.83      1205
           3       0.86      0.92      0.89      1184
           4       0.82      0.85      0.83      1202
           5       0.97      0.96      0.96      1211
           6       0.75      0.66      0.70      1218
           7       0.94      0.96      0.95      1159
           8       0.95      0.97      0.96      1197
           9       0.97      0.96      0.96      1203

    accuracy                           0.89     12000
   macro avg       0.89      0.89      0.89     12000
weighted avg       0.89      0.89      0.89     12000

              precision    recall  f1-score   support

           0       0.82      0.85      0.83      1000
           1       1.00      0.96      0.98      1000
           2       0.79      0.81      0.80      1000
           3       0.86 

#3. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred_val = clf.predict(X_val)
print(classification_report(y_val, y_pred_val))
y_pred_test = clf.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      1202
           1       1.00      0.97      0.98      1219
           2       0.79      0.82      0.80      1205
           3       0.87      0.91      0.89      1184
           4       0.77      0.83      0.80      1202
           5       0.97      0.96      0.97      1211
           6       0.75      0.60      0.67      1218
           7       0.94      0.94      0.94      1159
           8       0.96      0.97      0.96      1197
           9       0.95      0.96      0.95      1203

    accuracy                           0.88     12000
   macro avg       0.88      0.88      0.88     12000
weighted avg       0.88      0.88      0.88     12000

              precision    recall  f1-score   support

           0       0.82      0.85      0.83      1000
           1       0.99      0.96      0.97      1000
           2       0.77      0.80      0.78      1000
           3       0.87 

##4. k-Nearest Neighbors (kNN)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)

clf.fit(X_train, y_train)
y_pred_val = clf.predict(X_val)
print(classification_report(y_val, y_pred_val))
y_pred_test = clf.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.75      0.86      0.80      1202
           1       0.99      0.97      0.98      1219
           2       0.74      0.80      0.77      1205
           3       0.88      0.86      0.87      1184
           4       0.78      0.75      0.77      1202
           5       0.99      0.85      0.91      1211
           6       0.66      0.59      0.62      1218
           7       0.89      0.96      0.92      1159
           8       0.98      0.93      0.95      1197
           9       0.90      0.97      0.93      1203

    accuracy                           0.85     12000
   macro avg       0.86      0.85      0.85     12000
weighted avg       0.85      0.85      0.85     12000

              precision    recall  f1-score   support

           0       0.75      0.84      0.79      1000
           1       0.98      0.97      0.97      1000
           2       0.73      0.79      0.76      1000
           3       0.90 

# 5. Gradient Boosting Machines (GBM)


> Indented block



In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(X_train, y_train)
y_pred_val = clf.predict(X_val)
print(classification_report(y_val, y_pred_val))
y_pred_test = clf.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1202
           1       1.00      0.98      0.99      1219
           2       0.83      0.85      0.84      1205
           3       0.89      0.92      0.91      1184
           4       0.83      0.84      0.84      1202
           5       0.98      0.97      0.98      1211
           6       0.77      0.69      0.73      1218
           7       0.96      0.96      0.96      1159
           8       0.98      0.97      0.97      1197
           9       0.97      0.97      0.97      1203

    accuracy                           0.90     12000
   macro avg       0.90      0.90      0.90     12000
weighted avg       0.90      0.90      0.90     12000

              precision    recall  f1-score   support

           0       0.83      0.86      0.85      1000
           1       0.99      0.97      0.98      1000
           2       0.79      0.82      0.81      1000
           3       0.90 

#6. Exploring Neural Network more (Convolutional Neural Networks)

1.   Add Multiple Layers
2.   or we can try ResNet(if it's needed , probably NOT!!)
3.   check multiple opimizer also check gradient learning rates.
4.   add more data augmention.
5.   maxpool avgpool

add more visualizition for the report



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F




# Define complex network for FashionMNIST
class ComplexFashionMNISTNet(nn.Module):
    def __init__(self):
        super(ComplexFashionMNISTNet, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.avgpool(x)  # Global Average Pooling
        x = torch.flatten(x, 1)
        # Fully connected layers with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.RandomErasing()
])


model = ComplexFashionMNISTNet().to(device)



#enable GPU
model = ComplexFashionMNISTNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


#train
LOSSES = []
EPOCHES = []
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    LOSSES.append(running_loss)
    EPOCHES.append(epoch)

print('Finished Training')




[1,  2000] loss: 2.303
[1,  4000] loss: 2.302
[1,  6000] loss: 2.302
[1,  8000] loss: 2.301
[1, 10000] loss: 2.300
[1, 12000] loss: 2.299
[2,  2000] loss: 2.298
[2,  4000] loss: 2.295
[2,  6000] loss: 2.290
[2,  8000] loss: 2.280
[2, 10000] loss: 2.257
[2, 12000] loss: 2.205
[3,  2000] loss: 2.126


In [ ]:
model.eval()
from sklearn.metrics import classification_report, accuracy_score

y_pred= []
y_true=[]
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')

print(classification_report(y_true, y_pred, target_names=testset.classes, digits=3))


In [ ]:
# visualize activation in
import matplotlib.pyplot as plt
import numpy as np

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.conv3.register_forward_hook(get_activation('conv2'))
model.fc3.register_forward_hook(get_activation('fc3'))

images, _ = next(iter(testloader))
output = model(images)

def visualize_feature_maps(layer_activations):
    num_feature_maps = layer_activations.size(1)
    fig, axarr = plt.subplots(min(num_feature_maps, 4))
    for idx in range(min(num_feature_maps, 4)):
        feature_map = layer_activations[0][idx]
        axarr[idx].imshow(feature_map.cpu().numpy(), cmap="gray")
        axarr[idx].axis('off')
    plt.show()

visualize_feature_maps(activation['conv2'])

def visualize_fc_layer(layer_weights):
    plt.imshow(layer_weights.cpu().numpy(), cmap="coolwarm")
    plt.colorbar()
    plt.show()

visualize_fc_layer(model.fc3.weight)
